# Calculation of pi to 4000 decimals

In [1]:
import numpy as np
from scipy.signal import fftconvolve
import scipy.signal as sci
from scipy.fft import fft, ifft

In [2]:
def normalisation(x):
    for i in range(1, len(x) + 1):
        if i < len(x):
            x[-i-1] = x[-i-1] + x[-i] // 10
        x[-i] = x[-i] % 10
    return x

def division(x):
    for i in range(len(x)):
        x[i] = round(x[i], 1)
        if i < len(x) - 1:
            x[i+1] = x[i+1] + (x[i] % 1) * 10
        x[i] = x[i] // 1
    return x

def convolution_fft(x, y):
    n = len(x) + len(y) - 1
    fft_size = 2**int(np.ceil(np.log2(n)))
    X = np.fft.fft(x, fft_size)
    Y = np.fft.fft(y, fft_size)
    conv = np.fft.ifft(X * Y)
    conv = np.real(conv)
    return conv[:n]

In [3]:
np.set_printoptions(threshold=np.inf)
n = 8192
x = np.zeros(n)
one = np.full(n, 9)
one_d = np.zeros(n)
one_d[-1] = 1

threshold = 0.01
comp = False
i = 0
arr = []
j = 10

while comp == False:
    xn1 = x
#    a = np.convolve(x, x)
    a = convolution_fft(x, x)
    a[a < threshold] = 0
    b = np.insert(a[:n-1], 0, 0)
    c = normalisation(b)
    xn = 0.5 * (one - c + one_d)
    x = normalisation(division(xn))
    comp = (xn1[:4000] == x[:4000]).all()
    cond = (xn1[:j] == x[:j]).all()
    if cond == True:
        #print('j = {} reached, i = {}'.format(j, i))
        arr.append(i)
        j = j + 1
    i = i + 1

In [4]:
# Saving the decimals as individual characters
np.savetxt('sqrt2_fft.txt', x[:4000], delimiter=' ')

In [5]:
'''import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('fft_iter.csv')
data['i_np'] = arr

plt.plot(data['i_fft'], data['j'], ms = 0.5, label = 'FFT')
plt.plot(data['i_np'], data['j'], ms = 0.5, label = 'No FFT')

plt.xlabel('Number of iterations')
plt.ylabel('Reached decimal')
plt.legend()
plt.show()'''

"import matplotlib.pyplot as plt\nimport pandas as pd\n\ndata = pd.read_csv('fft_iter.csv')\ndata['i_np'] = arr\n\nplt.plot(data['i_fft'], data['j'], ms = 0.5, label = 'FFT')\nplt.plot(data['i_np'], data['j'], ms = 0.5, label = 'No FFT')\n\nplt.xlabel('Number of iterations')\nplt.ylabel('Reached decimal')\nplt.legend()\nplt.show()"

In [6]:
#data.to_csv('fft_iter.csv', index = False)

In [ ]:
# error estimation
sqrt2 = x

x = np.zeros(n)
threshold = 0.01
comp = False
i = 0
arr = []
en = sqrt2

while comp == False:
    en = normalisation(sqrt2 - x)
    xn1 = x
#    a = np.convolve(x, x)
    a = convolution_fft(x, x)
    a[a < threshold] = 0
    b = np.insert(a[:n-1], 0, 0)
    c = normalisation(b)
    xn = 0.5 * (one - c + one_d)
    x = normalisation(division(xn))
    comp = (xn1[:4000] == x[:4000]).all()
    en1 = normalisation(sqrt2 - x)
    r = np.sum(en1) / np.sum(en)
    arr.append((i, r))
    i = i + 1